In [ ]:
!pip install sklearn-genetic-opt

In [1]:
import subprocess
import numpy as np
import os

from sklearn.base import BaseEstimator, RegressorMixin

# Random Search one parameter optimization
Example, make sure it works before going to GA.  

Score computes with MSE loss and PSNR like:  
$$score = 1-MSE + PSNR/100$$

In [ ]:
class FDNeRF(BaseEstimator, RegressorMixin):  
    def __init__(self, n_layers=0):
        # Parameters should have same name as attributes
        self.n_layers = n_layers

    def fit(self, X, y=None):
        command = 'python run_nerf.py --config configs/lego.txt --n_iters 100'
        command += ' --n_layers '+ str(self.n_layers)
        command += ' --n_layers_fine '+str(self.n_layers)
        command += ' --i_print '+str(20)
        
        print(command)
        p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)

        (output, err) = p.communicate()  

        #This makes the wait possible
        p_status = p.wait()
        # print('Training done')

        return self
    
    def predict(self, X, y=None):
        try:
            getattr(self, "treshold_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")

        return self.result

    def score(self, X, y=None):
        train_dirs = sorted(os.listdir('logs'))
        train_dirs.remove('summaries')
        train_dirs_split = [x.split(' ')[1] for x in train_dirs]
        inexies = [i[0] for i in sorted(enumerate(train_dirs_split), key=lambda x:x[1])]
        mask = np.array(inexies) == max(inexies)
        file_name = np.array(train_dirs)[mask]

        last_train_file = 'logs/' + file_name[0] + '/loss_vs_time.pkl'

        data = np.load(last_train_file, allow_pickle=True)
        loss,psnr,time = data['losses'][-1], data['psnr'][-1], data['time'][-1]
        score = 1-loss + psnr/100
        print(f'score: {score}')
        return score

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
N = 3
X_train = np.linspace(1, 3, num=N)
X_test = np.linspace(1, 3, num=N)
dummy_y = [1 for i in range(N)]
tuned_params = {
    "n_layers" : np.array([1, 2, 8]).astype(int)
    }


gs = GridSearchCV(FDNeRF(), tuned_params, cv=[(slice(None), slice(None))])

# for some reason I have to pass y with same shape
# otherwise gridsearch throws an error. Not sure why.
# X and Y dosn't mean anything
# just makecompatable with sklearn
gs.fit(X_test, y=dummy_y)

gs.best_params_


# Genetic Optimization 
Parameter list:
- n_layers: 

In [2]:
class FDNeRF_GA(BaseEstimator, RegressorMixin):  
    def __init__(self, 
                 n_layers=0,
                 n_width=0,
                 n_layers_fine=0,
                 n_width_fine=0,
                 n_layers_time=0,
                 n_width_time=0,
                 multires=0,
                 multires_timenet=0,
                 ):
        # Parameters should have same name as attributes
        self.n_layers=n_layers
        self.n_width=n_width
        self.n_layers_fine=n_layers_fine
        self.n_width_fine=n_width_fine
        self.n_layers_time=n_layers_time
        self.n_width_time=n_width_time
        self.multires=multires
        self.multires_timenet=multires_timenet


    def fit(self, X, y=None):
        command = 'python run_nerf.py --config configs/lego.txt --n_iters 100'
        
        command += ' --n_layers '+ str(self.n_layers)
        command += ' --n_width '+str(self.n_width)
        
        command += ' --n_layers_fine '+str(self.n_layers_fine)
        command += ' --n_width_fine '+str(self.n_width_fine)

        command += ' --n_layers_fine '+str(self.n_layers_fine)
        command += ' --n_width_fine '+str(self.n_width_fine)
        
        command += ' --i_print '+str(20)
        
        print(command)
        p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)

        (output, err) = p.communicate()  

        #This makes the wait possible
        p_status = p.wait()
        # print('Training done')

        return self
    
    def predict(self, X, y=None):
        try:
            getattr(self, "treshold_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")

        return self.result

    def _compute_score(self,data):
        psnr = np.array(data['psnr']) 
        rev_time = np.array(data['time'])[::-1] # reversed
        _psnr = (psnr / rev_time).sum()

        loss = (1 - np.array(data['losses'])) / rev_time
        _loss = loss.sum()*10
        score =_psnr + _loss
        return score


    def score(self, X, y=None):
        train_dirs = sorted(os.listdir('logs'))
        train_dirs.remove('summaries')
        train_dirs_split = [x.split(' ')[1] for x in train_dirs]
        inexies = [i[0] for i in sorted(enumerate(train_dirs_split), key=lambda x:x[1])]
        mask = np.array(inexies) == max(inexies)
        file_name = np.array(train_dirs)[mask]

        last_train_file = 'logs/' + file_name[0] + '/loss_vs_time.pkl'

        data = np.load(last_train_file, allow_pickle=True)
        # loss,psnr,time = data['losses'][-1], data['psnr'][-1], data['time'][-1]
        # score = 1-loss + psnr/100
        score = self._compute_score(data)
        print(f'score: {score}')
        return score

In [3]:
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Categorical, Continuous,Integer
# https://sklearn-genetic-opt.readthedocs.io/en/stable/api/space.html
param_grid = {
    "n_layers" : Integer(1,8),
    "n_layers_fine" : Integer(1,8),
    
    }

evolved_estimator = GASearchCV(FDNeRF_GA(),
                    cv=[(slice(None), slice(None))],
                    # scoring='accuracy',
                    param_grid=param_grid,
                    population_size=3,
                    generations=3,
                    tournament_size=3,
                    elitism=True,
                    verbose=True)

In [4]:
N = 3
dummy_x = np.linspace(1, 3, num=N)
dummy_y = [1 for i in range(N)]

evolved_results = evolved_estimator.fit(dummy_x,dummy_y)

python run_nerf.py --config configs/lego.txt --n_iters 100 --n_layers 4 --n_width 0 --n_layers_fine 1 --n_width_fine 0 --n_layers_fine 1 --n_width_fine 0 --i_print 20
score: 10.48780272536377
python run_nerf.py --config configs/lego.txt --n_iters 100 --n_layers 3 --n_width 0 --n_layers_fine 1 --n_width_fine 0 --n_layers_fine 1 --n_width_fine 0 --i_print 20
score: 10.38626243462603
python run_nerf.py --config configs/lego.txt --n_iters 100 --n_layers 1 --n_width 0 --n_layers_fine 5 --n_width_fine 0 --n_layers_fine 5 --n_width_fine 0 --i_print 20
score: 11.487921050098445
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	3     	10.7873	0.497125   	11.4879    	10.3863    
python run_nerf.py --config configs/lego.txt --n_iters 100 --n_layers 3 --n_width 0 --n_layers_fine 1 --n_width_fine 0 --n_layers_fine 1 --n_width_fine 0 --i_print 20
score: 10.523050268722134
python run_nerf.py --config configs/lego.txt --n_iters 100 --n_layers 1 --n_width 0 --n_layers_fine 1 --n_width_fine 0 -

In [ ]:
evolved_estimator.best_params